In [0]:
from google.colab import drive
drive.mount('/content/drive')

*description of the columns**

SalePrice - the property's sale price in dollars. This is the target variable 

that you're trying to predict.

MSSubClass: The building class

MSZoning: The general zoning classification

LotFrontage: Linear feet of street connected to property

LotArea: Lot size in square feet

Street: Type of road access

Alley: Type of alley access

LotShape: General shape of property

LandContour: Flatness of the property

Utilities: Type of utilities available

LotConfig: Lot configuration

LandSlope: Slope of property

Neighborhood: Physical locations within Ames city limits

Condition1: Proximity to main road or railroad

Condition2: Proximity to main road or railroad (if a second is present)

BldgType: Type of dwelling

HouseStyle: Style of dwelling

OverallQual: Overall material and finish quality

OverallCond: Overall condition rating

YearBuilt: Original construction date

YearRemodAdd: Remodel date

RoofStyle: Type of roof
RoofMatl: Roof material

Exterior1st: Exterior covering on house

Exterior2nd: Exterior covering on house (if more than one material)

MasVnrType: Masonry veneer type

MasVnrArea: Masonry veneer area in square feet

ExterQual: Exterior material quality

ExterCond: Present condition of the material on the exterior

Foundation: Type of foundation

BsmtQual: Height of the basement

BsmtCond: General condition of the basement

BsmtExposure: Walkout or garden level basement walls

BsmtFinType1: Quality of basement finished area

BsmtFinSF1: Type 1 finished square feet

BsmtFinType2: Quality of second finished area (if present)

BsmtFinSF2: Type 2 finished square feet

BsmtUnfSF: Unfinished square feet of basement area

TotalBsmtSF: Total square feet of basement area

Heating: Type of heating

HeatingQC: Heating quality and condition

CentralAir: Central air conditioning

Electrical: Electrical system

1stFlrSF: First Floor square feet

2ndFlrSF: Second floor square feet

LowQualFinSF: Low quality finished square feet (all floors)

GrLivArea: Above grade (ground) living area square feet

BsmtFullBath: Basement full bathrooms

BsmtHalfBath: Basement half bathrooms

FullBath: Full bathrooms above grade

HalfBath: Half baths above grade

Bedroom: Number of bedrooms above basement level

Kitchen: Number of kitchens

KitchenQual: Kitchen quality
TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)

Functional: Home functionality rating

Fireplaces: Number of fireplaces

FireplaceQu: Fireplace quality

GarageType: Garage location

GarageYrBlt: Year garage was built

GarageFinish: Interior finish of the garage

GarageCars: Size of garage in car capacity

GarageArea: Size of garage in square feet

GarageQual: Garage quality

GarageCond: Garage condition

PavedDrive: Paved driveway

WoodDeckSF: Wood deck area in square feet

OpenPorchSF: Open porch area in square feet

EnclosedPorch: Enclosed porch area in square feet

3SsnPorch: Three season porch area in square feet

ScreenPorch: Screen porch area in square feet

PoolArea: Pool area in square feet

PoolQC: Pool quality

Fence: Fence quality

MiscFeature: Miscellaneous feature not covered in other categories

MiscVal: $Value of miscellaneous feature

MoSold: Month Sold

YrSold: Year Sold

SaleType: Type of sale

***SaleCondition: Condition of sale***









In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

 **READ THE TRAIN FILE OF HOUSE**

In [0]:
df_train=pd.read_csv("/content/drive/My Drive/Colab Notebooks/House Prices Advanced Regression Techniques/train.csv")

In [0]:
df_train.shape

In [0]:
df_train.head()

In [0]:
df_train.columns

*Here we gonna find the missing value percentage*

In [0]:
total=df_train.isnull().sum().sort_values(ascending=False)
percentage=(df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data=pd.concat([total,percentage],axis=1,keys=['total','percentage'])
missing_data.head(20)

**I THINK NONE THE VARIABLE ARE IMPORANT FOR PREDICT THE SALE PRICE SO ITS BETTER TO DELETE THESE VARIABLE**

In [0]:
df_train = df_train.drop((missing_data[missing_data['total'] > 1]).index,1)
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)

In [0]:
total=df_train.isnull().sum().sort_values(ascending=False)
percentage=(df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data=pd.concat([total,percentage],axis=1,keys=['total','percentage'])
missing_data.head(20)

**Analysing the sale price**

In [0]:
#descriptive statistics summary
df_train['SalePrice'].describe()

Very well... It seems that your minimum price is larger than zero. Excellent! 

In [0]:
#histogram
sns.distplot(df_train['SalePrice']);

Here i see some

**Deviate from the normal distribution.**

**Have appreciable positive skewness.**

**Show peakedness.**

In [0]:
#skewness and kurtosis
print("Skewness: %f" % df_train['SalePrice'].skew())
print("Kurtosis: %f" % df_train['SalePrice'].kurt())

i think some what we see it again haha!

In [0]:
#scatter plot grlivarea/saleprice
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

Hmmm... It seems that 'SalePrice' and 'GrLivArea' are really old friends, with a linear relationship.

In [0]:
#scatter plot totalbsmtsf/saleprice
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

'TotalBsmtSF' is also a great friend of 'SalePrice' but this seems a much more emotional relationship! Everything is ok and suddenly, in a strong linear (exponential?) reaction, everything changes. Moreover, it's clear that sometimes 'TotalBsmtSF' closes in itself and gives zero credit to 'SalePrice'.

In [0]:
#box plot overallqual/saleprice
var = 'OverallQual'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);

In [0]:
var = 'YearBuilt'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);

Although it's not a strong tendency, I'd say that 'SalePrice' is more prone to spend more money in new stuff than in old relics.

Note: we don't know if 'SalePrice' is in constant prices. Constant prices try to remove the effect of inflation. If 'SalePrice' is not in constant prices, it should be, so than prices are comparable over the years.

In summary
Stories aside, we can conclude that:

'GrLivArea' and 'TotalBsmtSF' seem to be linearly related with 'SalePrice'. Both relationships are positive, which means that as one variable increases, the other also increases. In the case of 'TotalBsmtSF', we can see that the slope of the linear relationship is particularly high.
'OverallQual' and 'YearBuilt' also seem to be related with 'SalePrice'. The relationship seems to be stronger in the case of 'OverallQual', where the box plot shows how sales prices increase with the overall quality.
We just analysed four variables, but there are many other that we should analyse. The trick here seems to be the choice of the right features (feature selection) and not the definition of complex relationships between them (feature engineering).

That said, let's separate the wheat from the chaff.

In [0]:
#correlation matrix
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(corrmat, vmax=.8, square=True);

At first sight, there are two red colored squares that get my attention. The first one refers to the 'TotalBsmtSF' and '1stFlrSF' variables, and the second one refers to the 'GarageX' variables. Both cases show how significant the correlation is between these variables. Actually, this correlation is so strong that it can indicate a situation of multicollinearity. If we think about these variables, we can conclude that they give almost the same information so multicollinearity really occurs. Heatmaps are great to detect this kind of situations and in problems dominated by feature selection, like ours, they are an essential tool.

Another thing that got my attention was the 'SalePrice' correlations. We can see our well-known 'GrLivArea', 'TotalBsmtSF', and 'OverallQual'

In [0]:
#saleprice correlation matrix
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.25)
f, ax = plt.subplots(figsize=(15, 10))
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

OverallQual', 'GrLivArea' and 'TotalBsmtSF' are strongly correlated with 'SalePrice'. Check!
'GarageCars' and 'GarageArea' are also some of the most strongly correlated variables. However, as we discussed in the last sub-point, the number of cars that fit into the garage is a consequence of the garage area. 'GarageCars' and 'GarageArea' are like twin brothers. You'll never be able to distinguish them. Therefore, we just need one of these variables in our analysis (we can keep 'GarageCars' since its correlation with 'SalePrice' is higher).
'TotalBsmtSF' and '1stFloor' also seem to be twin brothers. We can keep 'TotalBsmtSF' just to say that our first guess was right (re-read 'So... What can we expect?').
'FullBath'?? Really?
'TotRmsAbvGrd' and 'GrLivArea', twin brothers again. Is this dataset from Chernobyl?
Ah... 'YearBuilt'... It seems that 'YearBuilt' is slightly correlated with 'SalePrice'. Honestly, it scares me to think about 'YearBuilt' because I start feeling that we should do a little bit of time-series analysis to get this right. I'll leave this as a homework for you.

In [0]:
#scatterplot
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df_train[cols], size = 2.5)
plt.show();

Although we already know some of the main figures, this mega scatter plot gives us a reasonable idea about variables relationships.

One of the figures we may find interesting is the one between 'TotalBsmtSF' and 'GrLiveArea'. In this figure we can see the dots drawing a linear line, which almost acts like a border. It totally makes sense that the majority of the dots stay below that line. Basement areas can be equal to the above ground living area, but it is not expected a basement area bigger than the above ground living area (unless you're trying to buy a bunker).

The plot concerning 'SalePrice' and 'YearBuilt' can also make us think. In the bottom of the 'dots cloud', we see what almost appears to be a shy exponential function (be creative). We can also see this same tendency in the upper limit of the 'dots cloud' (be even more creative). Also, notice how the set of dots regarding the last years tend to stay above this limit (I just wanted to say that prices are increasing faster now).

**so here we must aware of the ouliers that should effect on saleprice**

**outliers detection**

**univariate analysis**

we will see only the sales column

In [0]:
#standardizing data
saleprice_scaled = StandardScaler().fit_transform(df_train['SalePrice'][:,np.newaxis]);
low_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][:10]
high_range= saleprice_scaled[saleprice_scaled[:,0].argsort()][-10:]
print('outer range (low) of the distribution:')
print(low_range)
print('\nouter range (high) of the distribution:')
print(high_range)

Low range values are similar and not too far from 0.

High range values are far from 0 and the 7.something values are really out of range.

For now, we'll not consider any of these values as an outlier but we should be careful with those two 7.something values.

**BIvariate analysis**

In [0]:
#bivariate analysis saleprice/grlivarea
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));



1.   The two values with bigger 'GrLivArea' seem strange and they are not following 
the crowd. We can speculate why this is happening. Maybe they refer to agricultural area and that could explain the low price. I'm not sure about this but I'm quite confident that these two points are not representative of the typical case. Therefore, we'll define them as outliers and delete them.
2. The two observations in the top of the plot are those 7.something observations that we said we should be careful about. They look like two special cases, however they seem to be following the trend. For that reason, we will keep them.


In [0]:
df_train.sort_values(by = 'GrLivArea', ascending = False)[:2]

In [0]:
df_train = df_train.drop(df_train[df_train['Id'] == 1299].index)
df_train = df_train.drop(df_train[df_train['Id'] == 524].index)

In [0]:
#bivariate analysis saleprice/grlivarea
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

Total square feet of basement area We can feel tempted to eliminate some observations (e.g. TotalBsmtSF > 3000) but I suppose it's not worth it. We can live with that, so we'll not do anything.

**Now it's time to go deep and understand how 'SalePrice' complies with the statistical assumptions that enables us to apply multivariate techniques**

**NORMALITY**

In [0]:
#histogram and normal probability plot
sns.distplot(df_train['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt)

Ok, 'SalePrice' is not normal. It shows 'peakedness', positive skewness and does not follow the diagonal line.

But everything's not lost. A simple data transformation can solve the problem. This is one of the awesome things you can learn in statistical books: in case of positive skewness, log transformations usually works well.

In [0]:
#applying log transformation
df_train['SalePrice'] = np.log(df_train['SalePrice'])

In [0]:
#transformed histogram and normal probability plot
sns.distplot(df_train['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt)

**Done! Let's check what's going on with 'GrLivArea'**

In [0]:
#histogram and normal probability plot
sns.distplot(df_train['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt)

It Look Like Some Skewness is there so we should apply the log transform  for this skewness

In [0]:
#applying log transformation
df_train['GrLivArea'] = np.log(df_train['GrLivArea'])

In [0]:
#histogram and normal probability plot
sns.distplot(df_train['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt)

**Done! Let's check what's going on with 'TotalBsmtSF'**

In [0]:
sns.distplot(df_train['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['TotalBsmtSF'], plot=plt)

Something that, in general, presents skewness.
A significant number of observations with value zero (houses without basement).
A big problem because the value zero doesn't allow us to do log transformations.
To apply a log transformation here, we'll create a variable that can get the effect of having or not having basement (binary variable). Then, we'll do a log transformation to all the non-zero observations, ignoring those with value zero. This way we can transform data, without losing the effect of having or not basement.

I'm not sure if this approach is correct. It just seemed right to me. That's what I call 'high risk engineering'.

In [0]:
#create column for new variable (one is enough because it's a binary categorical feature)
#if area>0 it gets 1, for area==0 it gets 0
df_train['HasBsmt'] = pd.Series(len(df_train['TotalBsmtSF']), index=df_train.index)
df_train['HasBsmt'] = 0 
df_train.loc[df_train['TotalBsmtSF']>0,'HasBsmt'] = 1

In [0]:
#transform data
df_train.loc[df_train['HasBsmt']==1,'TotalBsmtSF'] = np.log(df_train['TotalBsmtSF'])

In [0]:
#histogram and normal probability plot
sns.distplot(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], plot=plt)

**homoscedasticity**

In [0]:
#scatter plot
plt.scatter(df_train['GrLivArea'], df_train['SalePrice']);

Older versions of this scatter plot (previous to log transformations), had a conic shape (go back and check 'Scatter plots between 'SalePrice' and correlated variables (move like Jagger style)'). As you can see, the current scatter plot doesn't have a conic shape anymore. That's the power of normality! Just by ensuring normality in some variables, we solved the homoscedasticity problem.

Now let's check 'SalePrice' with 'TotalBsmtSF'.

In [0]:
#scatter plot
plt.scatter(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], df_train[df_train['TotalBsmtSF']>0]['SalePrice']);

We can say that, in general, 'SalePrice' exhibit equal levels of variance across the range of 'TotalBsmtSF'. Cool!

**dummy variables**

In [0]:
#convert categorical variable into dummy
df_train = pd.get_dummies(df_train)

In [0]:
df_train.shape

**FEATURE ENGINEERING**

Data Preparation

Let's now prepare the data and build the model.

In [0]:
# Lasso Regression
# Split
# Create matrix of all x features
X = df_train.drop(['SalePrice'], axis = 1)
# Create array of target variable
y = df_train['SalePrice']

In [0]:
# Split training data into train and test sets
from sklearn.model_selection import  train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.20,random_state=102)

Model Building and Evaluation

In [0]:
# Fit
# Import model
from sklearn.linear_model import Lasso
# Instantiate object
lasso = Lasso()
# Fit model to training data
lasso = lasso.fit(X_train, y_train)
# Predict
y_pred_lasso = lasso.predict(X_test)
# Score It
from sklearn import metrics
print('Linear Regression Performance')
print('MAE',metrics.mean_absolute_error(y_test, y_pred_lasso))
print('MSE',metrics.mean_squared_error(y_test, y_pred_lasso))
print('RMSE',np.sqrt(metrics.mean_squared_error(y_test, y_pred_lasso)))
print('R^2 =',metrics.explained_variance_score(y_test,y_pred_lasso))

In [0]:
# Lasso Coefficients
pd.set_option('display.float_format', lambda x: '%.2f' % x)
cdf = pd.DataFrame(data = lasso.coef_,index = X_train.columns, columns = ['Lasso Coefficients'])
# **RANDOM FOREST**
cdf.sort_values(by = 'Lasso Coefficients', ascending = False)

UNIVARIATE FEATURE SELECTION

In [0]:
# Select top 20% of features
# Create matrix of x features
X = df_train.drop(['SalePrice'],axis = 1)
# Create array of target variable y
y =df_train['SalePrice']
# Feature Selector
# Import
from sklearn.feature_selection import SelectPercentile, f_regression
# Instantiate object
selector_f = SelectPercentile(f_regression, percentile=10)
# Fit and transform
x_best = selector_f.fit_transform(X, y)

Pareto Approach

In choosing the percentile cutoff, I took a Pareto 80/20 approach and selected the top 20% best, associated features. However, I'll review the F-score and p-values to confirm that 20% is an appropriate percentile to exclude a feature from participating in the learning process.

In [0]:
support = np.asarray(selector_f.get_support())

# Supress displaying long numbers in scientific notation
#pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Enable scientific notation
pd.set_option('display.float_format', '{:.2e}'.format)

# Column names of top 20%
features = np.asarray(X.columns.values)
features_with_support = features[support]
# print('Top 20% of the best, associated features to SalePrice\n',columns_with_support)
# print('Number of Features:', len(columns_with_support))

#f-scores of top 20%
fscores = np.asarray(selector_f.scores_)
fscores_with_support = fscores[support]

# p-values of top 20%
pvalues = np.asarray(selector_f.pvalues_)
pvalues_with_support = pvalues[support]

# Dataframe of top 20%
top20 = pd.DataFrame({'F-score':fscores_with_support,
                      'p-value':pvalues_with_support},
                     index = features_with_support)
# top20.index.name = 'Feature'
print('Top 20% best associated features to SalePrice\nNumber of features:',len(features_with_support))
print(top20.sort_values(by = 'p-value', ascending = 'True'))

In [0]:
best_feat = df_train[features_with_support]
corr =best_feat.corr() # We already examined SalePrice correlations
plt.figure(figsize=(18, 12))

sns.heatmap(corr[(corr >= 0.7) | (corr <= -0.7)], 
            cmap='coolwarm', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

Feature to Feature Correlation

There are a few variables that are highly correlated (correlation >.80) with one another . The features representing redundant information and are less correlated with SalePrice can also be removed for further dimentionality reduction.

In [0]:
# Correlations to SalePrice
from scipy import stats
print('Correlation to SalePrice')
print('GrLivArea:',stats.pearsonr(best_feat['GrLivArea'],df_train['SalePrice'])[0])
print('TotRmsAbvGrd:',stats.pearsonr(best_feat['TotRmsAbvGrd'],df_train['SalePrice'])[0])
print('--'*40)
print('GarageCars:',stats.pearsonr(best_feat['GarageCars'],df_train['SalePrice'])[0])
print('GarageArea:',stats.pearsonr(best_feat['GarageArea'],df_train['SalePrice'])[0])

Removing Duplicated Features

GrLivArea vs TotRmsAbvGrd
These features are highly correlated with a correlation of 0.83. Of these two variables, TotRmsAbvGrd will be removed because it has a lower correlation with SalePrice.

Garage Cars vs Garage Area
Garage Cars and GarageArea are also highly correlated as indicated by their correlation coefficient of 0.88. GarageArea will be removed as it has a lower correlation with our target variable, SalePrice.

In [0]:
# Remove redundant features
best_feat = best_feat.drop(['TotRmsAbvGrd','GarageArea'], axis = 1)
best_feat.columns

In [0]:
# Random Forest Regression with Best Features
# Split
# Create matrix of best x features
X_best = df_train[['OverallQual', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'TotalBsmtSF',
       '1stFlrSF', 'GrLivArea', 'FullBath', 'Fireplaces', 'GarageCars',
       'Neighborhood_NridgHt', 'ExterQual_Ex', 'ExterQual_Gd', 'ExterQual_TA',
       'Foundation_PConc', 'HeatingQC_Ex', 'KitchenQual_Ex', 'KitchenQual_TA',
       'SaleType_New', 'SaleCondition_Partial']]

# Create array of target variable
y = df_train['SalePrice']

# Split training data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_best,y, test_size = .20,random_state = 102)

# Fit
from sklearn.ensemble import RandomForestRegressor
rforest = RandomForestRegressor(n_estimators = 300, random_state =102) 
rforest.fit(X_best,y)

# Predict
y_pred_rforest = rforest.predict(X_test)

# Score It
from sklearn import metrics
print('Random Forest Regression Performance')
print('MAE',metrics.mean_absolute_error(y_test, y_pred_rforest))
print('MSE',metrics.mean_squared_error(y_test, y_pred_rforest))
print('RMSE',np.sqrt(metrics.mean_squared_error(y_test, y_pred_rforest)))
print('R^2 =',metrics.explained_variance_score(y_test,y_pred_rforest))

WITH TEST DATA

In [0]:
# Load test data
df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/House Prices Advanced Regression Techniques/test.csv')

In [0]:
total=df_test.isnull().sum().sort_values(ascending=False)
percentage=(df_test.isnull().sum()/df_test.isnull().count()).sort_values(ascending=False)
missing_data=pd.concat([total,percentage],axis=1,keys=['total','percentage'])
missing_data.head(20)

In [0]:
df_test= df_test.drop((missing_data[missing_data['total'] >0]).index,1)

In [0]:
total=df_test.isnull().sum().sort_values(ascending=False)
percentage=(df_test.isnull().sum()/df_test.isnull().count()).sort_values(ascending=False)
missing_data=pd.concat([total,percentage],axis=1,keys=['total','percentage'])
missing_data.head(20)

In [0]:
#convert categorical variable into dummy
df_test = pd.get_dummies(df_test)

In [0]:
# Create matrix of x features for training data
X_train2 = df_train[['OverallQual', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'TotalBsmtSF',
       '1stFlrSF', 'GrLivArea', 'FullBath', 'Fireplaces', 'GarageCars',
       'Neighborhood_NridgHt', 'ExterQual_Ex', 'ExterQual_Gd', 'ExterQual_TA',
       'Foundation_PConc', 'HeatingQC_Ex', 'KitchenQual_Ex', 'KitchenQual_TA',
       'SaleType_New', 'SaleCondition_Partial']]

# Create target variable array for training data
y_train2 = df_train['SalePrice']

# Create matrix of x features for test data
X_test2 = df_train[['OverallQual', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'TotalBsmtSF',
       '1stFlrSF', 'GrLivArea', 'FullBath', 'Fireplaces', 'GarageCars',
       'Neighborhood_NridgHt', 'ExterQual_Ex', 'ExterQual_Gd', 'ExterQual_TA',
       'Foundation_PConc', 'HeatingQC_Ex', 'KitchenQual_Ex', 'KitchenQual_TA',
       'SaleType_New', 'SaleCondition_Partial']]

# There is no target variable array for test data

# Confirm data shapes
print('Data Shapes')
print('x_train shape', X_train2.shape)
print('y_train shape',y_train2.shape)
print('x_test shape', X_test2.shape)

# Fit Random Forest to training data
from sklearn.ensemble import RandomForestRegressor
rforest = RandomForestRegressor(n_estimators = 300, random_state = 0) 
rforest.fit(X_train2,y_train2)

# Predict using test data
y_pred_rforest2 = rforest.predict(X_test2)

In [0]:
submission = pd.DataFrame({"Id": df_test["Id"],"SalePrice": y_pred_rforest2})

In [0]:
submission.head()

**SO HERE WE PERDICT THE HOUSE PRICE AS PER THE HOUSE PERSQUARE YARD**

In [0]:
submission.to_csv('HousingSubmissionbb.csv', index=False)